# Домашнее задание по теме "Распознавание языка" 

#### Соня Дурнева, БКЛ-151

### Первый метод: частотные слова

Сначала при помощи пакета wikipedia скачаем по сто текстов для каждого из выбранных языков.

In [37]:
def get_texts_for_lang(lang, n=10):
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue
        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))
    return wiki_content

In [38]:
import wikipedia 
langs = ('kk', 'uk', 'be', 'fr', 'en', 'de')

wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Skipping page Бестөбе ауылдық округі
Skipping page Тұщықұдық
kk 98
Skipping page Амальтея (значення)
Skipping page Каупер
Skipping page Алфей
Skipping page 1384 (значення)
Skipping page Семей (значення)
Skipping page Базарська сільська рада
uk 94
Skipping page Ласінцы
Skipping page Масюкоўшчына
Skipping page Янавіцкі сельсавет
be 97
Skipping page Fernando Marías
Skipping page Marie de Brabant
Skipping page Lorier
fr 97
Skipping page Cunderdin, Western Australia
Skipping page Hiroyuki Kobayashi (baseball)
Skipping page Diecast
Skipping page Dalzell
Skipping page Center Street Historic District
Skipping page Ionis
en 94
Skipping page Sven Pettersson (Ballsportler)
Skipping page Tvrdy
Skipping page SLE
Skipping page RWK
Skipping page Emerson
Skipping page Leun (Begriffsklärung)
Skipping page Schimmelpenninck
Skipping page George Perkins
Skipping page Newbigging
Skipping page Kaula
de 90


Теперь на материале скачанных текстов создадим частотные списки для каждого из языков. Оставим топ-80 слов, не сопровождая их частотностью: при определении языка нам понадобятся только сами слова.

In [39]:
import codecs
import collections
import sys
from nltk import word_tokenize
import string
from string import punctuation

def tokenize(text):
    punct_extended = string.punctuation + '«»—…“”'
    return [word.lower() for word in word_tokenize(text) if len(word) > 1 
                                                    and not all([ch in punct_extended for ch in word])]

word_freq_dicts = collections.defaultdict(lambda: 0)
for lang in langs:
    corpus = wiki_texts[lang]
    freqs = collections.defaultdict(lambda: 0)
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs[word] += 1
    word_freq_dicts[lang] = sorted(freqs, key=lambda w: freqs[w], reverse=True)[:80]

 Чтобы убедиться, что всё в порядке, выведем топ частотных слов французского языка.

In [40]:
print(word_freq_dicts['fr'])

['de', 'la', 'et', 'le', 'en', 'du', 'les', 'des', 'est', 'un', 'dans', 'par', 'au', 'une', 'sur', 'pour', 'il', 'qui', 'avec', 'sont', 'plus', 'que', 'son', 'elle', 'se', 'portail', 'sa', 'ou', 'paris', 'été', 'aux', 'ce', 'deux', 'ses', 'comme', 'ville', 'siècle', 'pays', 'nouvelle-zélande', 'cette', 'entre', 'mais', 'années', 'ont', 'france', 'pas', "d'un", "d'une", 'ne', 'barbara', 'juin', 'novembre', 'fait', 'timbre', 'premier', 'après', 'janvier', 'avril', 'mai', 'également', 'depuis', 'où', 'dont', 'mars', 'français', 'trois', 'février', 'septembre', 'sud', 'frf', 'décembre', 'on', 'très', 'première', 'était', 'the', 'même', 'argenteuil', 'aussi', 'juillet']


Напишем функцию *lang_detect_word( )*, которая будет принимать на вход текст и определять его язык. 

In [41]:
import operator
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def lang_detect_word(text):
    text = tokenize(text)
    options = {}
    for lang in word_freq_dicts:
        options[lang] = len(intersection(text, word_freq_dicts[lang]))
    return(max(options.items(), key=operator.itemgetter(1))[0])

Проверим, как работает функция на двух текстах – на белорусском языке и на английском.

In [42]:
test_text1 = '- Добры дзень, сястра! - Добры дзень, Віця! - Я бачу, ты адна дома. А дзе бацькі? - Да бабулі паехалі абодва. Вернуцца толькі ўвечары. - А чаму яны мяне не папярэдзілі, што паедуць? Я, можа быць, і сам бы бабулю дачуўся. - А яны спачатку не плянавалі ехаць. Потым бабуля патэлефанавала, прасіла, каб яны прыехалі, дапамаглі ей. Ей трэба, каб тата старую дзіцячую ложачак сабраў і адвез, куды трэба. - Ложачак? Навошта? - А, Віця, ты ж яшчэ не ведаеш! У цеткі Іны і дзядзькі Вовы нарадзілася маленькая дзяўчынка, сення рана раніцай. - Ух, ты! - Грошай у іх цяпер мала, вось наша бабуля і вырашыла ім дапамагчы. Знайшла старую ложачак і дзіцячую ванначку, розныя рэчы для маляняці. Усе яны ў добрым стане. Цяпер хоча пераправіць іх цетцы Іне. - А як назавуць дзяўчынку? - Яшчэ не вырашылі, але хочуць, накшталт, Полинкой назваць. - Добра. Паліна - прыгожае імя. Цяпер у нас з\'явілася траюрадная сястрычка! Подробнее - на Znanija.com - https://znanija.com/task/6101254#readmore'
test_text2 = 'Apple trees are large if grown from seed. Generally apple cultivars are propagated by grafting onto rootstocks, which control the size of the resulting tree. There are more than 7,500 known cultivars of apples, resulting in a range of desired characteristics. Different cultivars are bred for various tastes and uses, including cooking, eating raw and cider production. Trees and fruit are prone to a number of fungal, bacterial and pest problems, which can be controlled by a number of organic and non-organic means. In 2010, the fruit\'s genome was sequenced as part of research on disease control and selective breeding in apple production. Worldwide production of apples in 2014 was 84.6 million tonnes, with China accounting for 48% of the total.'
print(lang_detect_word(test_text1))
print(lang_detect_word(test_text2))

be
en


### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [43]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [44]:
freqs = collections.defaultdict(lambda: 0)


ngram_freq_dicts = collections.defaultdict(lambda: 0)
for lang in langs:
    corpus = wiki_texts[lang]
    freqs = collections.defaultdict(lambda: 0)
    for article in corpus:
        for ngram in make_ngrams(article.replace('\n', '').lower()):
            freqs[ngram] += 1
    ngram_freq_dicts[lang] = sorted(freqs, key=lambda n: freqs[n], reverse=True)[:80]

Напишем функцию *lang_detect_ngram( )*, аналогичную прошлой, но определяющей язык на основании пересечений с частотным словарём символьных n-грамм.

In [45]:
import operator

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def lang_detect_ngram(text):
    text = make_ngrams(text)
    options = {}
    for lang in ngram_freq_dicts:
        options[lang] = len(intersection(text, ngram_freq_dicts[lang]))
    return(max(options.items(), key=operator.itemgetter(1))[0])

Проверим работу функции на тех же текстах.

In [46]:
test_text1 = '- Добры дзень, сястра! - Добры дзень, Віця! - Я бачу, ты адна дома. А дзе бацькі? - Да бабулі паехалі абодва. Вернуцца толькі ўвечары. - А чаму яны мяне не папярэдзілі, што паедуць? Я, можа быць, і сам бы бабулю дачуўся. - А яны спачатку не плянавалі ехаць. Потым бабуля патэлефанавала, прасіла, каб яны прыехалі, дапамаглі ей. Ей трэба, каб тата старую дзіцячую ложачак сабраў і адвез, куды трэба. - Ложачак? Навошта? - А, Віця, ты ж яшчэ не ведаеш! У цеткі Іны і дзядзькі Вовы нарадзілася маленькая дзяўчынка, сення рана раніцай. - Ух, ты! - Грошай у іх цяпер мала, вось наша бабуля і вырашыла ім дапамагчы. Знайшла старую ложачак і дзіцячую ванначку, розныя рэчы для маляняці. Усе яны ў добрым стане. Цяпер хоча пераправіць іх цетцы Іне. - А як назавуць дзяўчынку? - Яшчэ не вырашылі, але хочуць, накшталт, Полинкой назваць. - Добра. Паліна - прыгожае імя. Цяпер у нас з\'явілася траюрадная сястрычка! Подробнее - на Znanija.com - https://znanija.com/task/6101254#readmore'
test_text2 = 'Apple trees are large if grown from seed. Generally apple cultivars are propagated by grafting onto rootstocks, which control the size of the resulting tree. There are more than 7,500 known cultivars of apples, resulting in a range of desired characteristics. Different cultivars are bred for various tastes and uses, including cooking, eating raw and cider production. Trees and fruit are prone to a number of fungal, bacterial and pest problems, which can be controlled by a number of organic and non-organic means. In 2010, the fruit\'s genome was sequenced as part of research on disease control and selective breeding in apple production. Worldwide production of apples in 2014 was 84.6 million tonnes, with China accounting for 48% of the total.'
print(lang_detect_ngram(test_text1))
print(lang_detect_ngram(test_text2))

be
en


### Тестирование

Теперь узнаем, с какой точностью определяется язык при использовании этих методов. Для этого загрузим новые тексты на тех же языках.

In [47]:
test_texts = {}
for lang in langs:
    test_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(test_texts[lang]))

kk 100


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Skipping page Кунжек
Skipping page Гур'євський район
Skipping page Кулаковка
Skipping page Еберт
Skipping page Діва (значення)
uk 95
Skipping page Іваноўшчына
Skipping page Зарачаны
Skipping page Духан
Skipping page Ванкевіч
Skipping page Гродзенскі павет (значэнні)
Skipping page Ульянаўка
Skipping page Мікалай Стэфанавіч Якушаў
Skipping page Беседзь (значэнні)
Skipping page Сконэ
be 91
Skipping page Uminski
Skipping page Arcole (Italie)
Skipping page West
fr 97
Skipping page Edward Edwards (actor)
Skipping page Hugh McFadden (poet)
Skipping page Surcharge
Skipping page Sepon
Skipping page Henryk Grabowski (athlete)
Skipping page Robert Wilkin
Skipping page Morozov
en 93
Skipping page Motorette
Skipping page Nüssler
Skipping page Vivat!
Skipping page Kloster Fontaine
Skipping page Procopé
Skipping page John L. Mack
Skipping page Fountain
Skipping page Santa Brigida
Skipping page Strength Through Joy
Skipping page Fullen
de 90


In [48]:
def test(texts_dict):
    errors_word_freq = 0
    errors_ngram_freq = 0
    texts = 0
    for lang in texts_dict:
        corpus = texts_dict[lang]
        for article in corpus:
            texts += 1
            if lang != lang_detect_word(article):
                errors_word_freq += 1
            if lang != lang_detect_ngram(article):
                errors_ngram_freq += 1
    print('word frequency method error rate:', errors_word_freq/texts)
    print('ngram frequency method error rate:', errors_ngram_freq/texts)

In [49]:
test(test_texts)

word frequency method error rate: 0.019434628975265017
ngram frequency method error rate: 0.007067137809187279


Видим, что точнее работает метод определения языка на основании частотного списка n-грамм – его error rate меньше, чем у метода определения языка по частотным словам.